# Benchmark ELEP for phase picking surface events

We will test the ELEP workflow to phase picks the SU events picked by the PNSN.
This notebooks focuses on the multiband workflow (a bit complicated!)

In [2]:
# import sys
# sys.path.append('/data/wsd01/pnwstore/')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import obspy
from obspy.core import UTCDateTime
from obspy.clients.fdsn.client import Client

import scipy
from scipy import optimize
from scipy.optimize import curve_fit
from geopy import distance
from datetime import datetime
from utils import *
import torch
import gc
plt.rcParams.update({'font.size': 10})


import seisbench.models as sbm
device = torch.device("cpu")

from ELEP.elep.ensemble_statistics import ensemble_statistics
from ELEP.elep.ensemble_coherence import ensemble_semblance 
from ELEP.elep.ensemble_learners import ensemble_regressor_cnn
from ELEP.elep import mbf, mbf_utils
from ELEP.elep import trigger_func

from ELEP.elep.mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from ELEP.elep.mbf import MB_filter

In [3]:
# define clients to download the station data
# client = WaveformClient() # we ignore PNWdatastore for now
client2 = Client('IRIS')

t_before = 15 #number of seconds before pick time
# t_after = 15 #number of seconds after pick time
t_before_raw = 1200 #number of seconds before pick time before removing instrumental response
# t_after_raw = 1200 #number of seconds after pick time before removing instrumental response
t_around = 5 # picking error around the reference pick time.
fs = 40 #sampling rate that all waveforms are resampled to
window = 150 #window length of the signal (this will help with phase picking with EqT next). 
# Use 150 seconds @ 40 Hz gives 6001 points. 
pr = 98 #percentile
thr = 5 #SNR threshold
station_distance_threshold = 25
pi = np.pi
v_s = 1000 #shear wave velocity at the surface

# range of dates that we are looking at
t_beginning = UTCDateTime(2001,1,1,0,0,0) 
t_end = UTCDateTime(2023,8,2,23,59)
twin=6000

smooth_length = 5 # constant for smoothing the waveform envelopes
low_cut = 1 #low frequency threshold
high_cut = 12 #high frequency threshold
az_thr = 1000 #threshold of distance in meters from source location
step = 100 #step every 100 m
t_step = 1 #step every second
ratio = 5.6915196 #used to define the grid 
colors = list(plt.cm.tab10(np.arange(10)))*3
radius = 6371e3 # radius of the earth

In [4]:
# download models
pretrain_list = ["pnw","ethz","instance","scedc","stead","geofon","neic"]
pn_pnw_model = sbm.EQTransformer.from_pretrained('pnw')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
pn_neic_model = sbm.EQTransformer.from_pretrained("neic")
list_models = [pn_pnw_model,pn_ethz_model,pn_scedc_model,pn_neic_model,pn_geofon_model,pn_stead_model,pn_instance_model]


In [5]:
pn_pnw_model.to(device);
pn_ethz_model.to(device);
pn_scedc_model.to(device);
pn_neic_model.to(device);
pn_geofon_model.to(device);
pn_stead_model.to(device);
pn_instance_model.to(device);

## volcano info

In [6]:
#this data includes all stations within 50km of each volcano and the lat, lon, elev of each station
df = pd.read_csv('../data/station/Volcano_Metadata_50km.csv')

### 3. Surface Event Data from PNSN

Read files straight from PNSN data base. Extract station and network code, phase pick time (seen as start_time). this should be converted to pick_time0.

In [7]:
f1 = pd.read_csv("../data/events/su_picks.txt",sep="|")
f1.head()
print(f1.keys())

Index(['date', 'net', 'sta', 'location', 'seedchan', 'iphase', 'quality',
       'orid', 'etype', 'evid'],
      dtype='object')


In [8]:
# clean up the spaces in the file
format='%Y/%m/%d %H:%M:%S'
test=f1["date"].values.tolist()
start_time_temp = [  datetime.strptime(x.strip(),'%Y/%m/%d %H:%M:%S') for x in f1["date"].values.tolist()]
# # Ignore events prior to t_beginning
ik=np.where(np.array(start_time_temp)>datetime(2001,1,1))[0][0]

# select only net, sta, evid, startime for event past the start date.

start_time = start_time_temp[ik:]
net=[ x.strip() for x in f1["net"].values.tolist()][ik:]
sta=[ x.strip() for x in f1["sta"].values.tolist()][ik:]
evt_id=[ x for x in f1["orid"].values.tolist()][ik:]
all_stas=set(sta)


f2=pd.DataFrame({"date":start_time,"net":net,"sta":sta,"evt_id":evt_id})
f2.head()

,date,net,sta,evt_id
0,2001-01-05 13:35:05,UW,YEL,801538
1,2001-01-05 14:37:57,UW,YEL,801553
2,2001-01-18 21:25:51,UW,SEP,807903
3,2001-01-18 21:25:52,UW,YEL,807903
4,2001-01-18 21:25:52,UW,SHW,807903


## Multiband & Semblance parameters

In [9]:
paras_semblance = {'dt':0.025, 'semblance_order':4, 'window_flag':True, 
                   'semblance_win':0.5, 'weight_flag':'max'}
p_thrd, s_thrd = 0.01, 0.05

fqmin = low_cut
fqmax = high_cut
dt = 0.025; fs = 40
nfqs = 10
nt = 6000; nc = 3
fq_list = make_LogFq(fqmin, fqmax, dt, nfqs)
coeff_HP, coeff_LP = rec_filter_coeff(fq_list, dt)
MBF_paras = {'f_min':fqmin, 'f_max':fqmax, 'nfqs':nfqs, 'frequencies':fq_list, 'CN_HP':coeff_HP, 'CN_LP':coeff_LP, \
    'dt':dt, 'fs':fs, 'nt':nt, 'nc':nc, 'npoles': 2}

## Full stack

In [10]:
pick_error = np.zeros(len(evt_id))
pick_error_mbf = np.zeros(len(evt_id))
maxsnr = np.zeros(len(evt_id))
whatvolcano = np.array(len(evt_id))
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('../plots/MLPicks_MBF_test.pdf')
for n in range(len(evt_id)): 
    # if start_time[n]<datetime(2020,1,1):continue   
    event_ID = str(evt_id[n])
    otime = UTCDateTime(start_time[n])
    networks=net[n]
    stations=sta[n]
    if sta[n]=="LON" or sta[n]=="LO2":continue
    # print(net[n],sta[n])
    try:
        associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
    except: 
        pass
    # if associated_volcano!="Mt_Rainier":continue
            
#################### WAVEFORM DOWNLOAD #######################
    #Download all waveforms for that event based on stations and times
    bulk=[[net[n], sta[n], '*', '*H*', otime-t_before_raw, otime+t_before_raw]]
    try:
        st = client2.get_waveforms_bulk(bulk)
        st = resample(st,fs)  #resampling the data to 40Hz for each trace
        
        evt_data = obspy.Stream()

        # #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
        SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
        for i,ii in enumerate(st):
            network = ii.stats.network
            station = ii.stats.station
            ii.detrend(type = 'demean')
            ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
            cha = ii.stats.channel
            starttime = ii.stats.starttime
            signal_window = ii.copy()
            noise_window = ii.copy()
            # trim the data and noise window to exactly 6000 points
            signal_window.trim(otime - t_before, otime - t_before + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
            noise_window.trim(otime - window -t_before, otime - t_before) # noise window of the same length
            if not len(signal_window.data) or not len(signal_window.data): continue
            # print(len(signal_window),len(noise_window.data))
            if not np.max(np.percentile(np.abs(signal_window.data),pr)):continue
            if len(noise_window.data)==0:continue
            snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                            / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
            max_amp_time = np.argmax(noise_window.data)/fs
            # print(snr,max_amp_time)
            # signal_window.plot()x
            # if snr<thr: 
            #     st.remove(ii)
            #     continue
            # start the time axis 15 seconds before the pick time of the first reference station
            t = signal_window.times()[:-1] #
            t_diff[network+'.'+station] = starttime-otime 
            stas.append(ii.stats.station)
            nets.append(ii.stats.network)
            SNR.append(snr)
            SNR_weight.append(int(snr))
            # no_weight.append(1)
            evt_data.append(signal_window)
            
        sta_available,ind = np.unique(np.array(stas),return_index=True)
        sta_available=sta_available[np.argsort(ind)]
        if len(sta_available)==0:continue

        print(event_ID,otime)
        print(associated_volcano)
        bigS = np.zeros(shape=(len(sta_available),3,6000))
        stas=[]
        # print(sta_available)
        # print(evt_data)
        for i in range(len(sta_available)):
            stream = evt_data.select(station=sta_available[i])
            # print(i,stream)
            if len(stream[0].data)<6000:continue
            # print("original stream")
            # print(stream)
            if len(stream)<3:
                # copy stream to 2 components, zero the missing data.
                tr3 = stream[0].copy() # assumed to be the vertical
                tr2 = stream[0].copy();tr2.stats.channel=stream[0].stats.channel[0:2]+"N"
                tr1 = stream[0].copy();tr1.stats.channel=stream[0].stats.channel[0:2]+"E"
                tr1.data=np.zeros(len(stream[0].data))
                tr2.data=np.zeros(len(stream[0].data))
                stream=obspy.Stream(traces=[tr1,tr2,tr3])
            # convert Stream into seisbench-friendly array    
            # fill in big array and order data ZNE
            bigS[i,0,:]=stream[2].data[:-1]
            bigS[i,1,:]=stream[1].data[:-1]
            bigS[i,2,:]=stream[0].data[:-1]
            stas.append(sta_available[i])


        
        # print("put data in numpy array")

        nwin,twin=bigS.shape[1],bigS.shape[-1]
        nsta = len(sta_available)
        # print("There are %i stations available"%nsta)

        # allocating memory for the ensemble predictions
        # 7 pre-trained models, nsta stations for batch, 6000 input points
        batch_pred =np.zeros(shape=(7,nsta,6000)) 
        batch_pred_mbf =np.zeros(shape=(7,nsta,6000))


        # evaluate
        pn_pnw_model.eval()
        pn_ethz_model.eval()
        pn_scedc_model.eval()
        pn_neic_model.eval()
        pn_geofon_model.eval()
        pn_stead_model.eval()
        pn_instance_model.eval()

        ######### Broadband workflow ################
        crap2  = bigS.copy()
        crap2 -= np.mean(crap2,axis=-1,keepdims=True) # demean data
        # original use std norm
        data_std = crap2 / np.std(crap2) + 1e-10
        # could use max data
        mmax= np.max(np.abs(crap2), axis=-1, keepdims=True)
        data_max = np.divide(crap2 ,mmax,out=np.zeros_like(crap2),where=mmax!=0)
        data_tt = torch.Tensor(data_max)
        # batch predict picks.
        _torch_pred_1 = pn_pnw_model(data_tt.to(device))
        _torch_pred_2 = pn_ethz_model(data_tt.to(device))
        _torch_pred_3 = pn_scedc_model(data_tt.to(device))
        _torch_pred_4 = pn_neic_model(data_tt.to(device))
        _torch_pred_5 = pn_geofon_model(data_tt.to(device))
        _torch_pred_6 = pn_stead_model(data_tt.to(device))
        _torch_pred_7 = pn_instance_model(data_tt.to(device))
        # extract P pdf

        # print("now predicting on BB")
        batch_pred[0,:, :] = _torch_pred_1[1].detach().cpu().numpy()[:, :]
        batch_pred[1,:, :] = _torch_pred_2[1].detach().cpu().numpy()[:, :]
        batch_pred[2,:, :] = _torch_pred_3[1].detach().cpu().numpy()[:, :]
        batch_pred[3,:, :] = _torch_pred_4[1].detach().cpu().numpy()[:, :]
        batch_pred[4,:, :] = _torch_pred_5[1].detach().cpu().numpy()[:, :]
        batch_pred[5,:, :] = _torch_pred_6[1].detach().cpu().numpy()[:, :]
        batch_pred[6,:, :] = _torch_pred_7[1].detach().cpu().numpy()[:, :]

        


        ############# Multi-band Workflow ########
        windows_std = np.zeros(shape=(nsta, nfqs, 3, twin), dtype= np.float32)
        windows_max = np.zeros(shape=( nsta, nfqs, 3, twin), dtype= np.float32)
        _windows = bigS.copy();#np.zeros(shape=(nsta, 3, twin), dtype= np.float32)
        _windows_mb = np.zeros(shape=(nsta, 3, nfqs, twin), dtype= np.float32)

        # MB filter
        for ista in range(nsta): # loop over stations, it should be one in this benchmark test
            for icha in range(3): # loop over channel, there should be 3 channels total
                _windows_mb[ista, icha, :, :] = MB_filter(_windows[ista, icha], MBF_paras)
        _windows_mb = _windows_mb.swapaxes(1, 2)
        # print("Shape of the multi-band data array: ",_windows_mb.shape)
        # normalize the multi band data
        for ista in range(nsta):
            for ifreq in range(nfqs):
                # original use std norm
                windows_std[ista, ifreq,:, :] = _windows_mb[ista, ifreq, :]  \
                    / np.std(_windows_mb[ista, ifreq, :]) + 1e-10
                # others use max norm
                mmax=np.max(np.abs(_windows_mb[ista, ifreq,:, :]), axis=-1, keepdims=True)
                windows_max[ista, ifreq,:, :] = np.divide(_windows_mb[ista, ifreq,:, :] \
                    ,mmax,out=np.zeros_like(_windows_mb[ista, ifreq, :]),where=mmax!=0)


        # print(f"Window data shape: {data_std.shape}"))


        # print("now predicting on MBF")
        batch_pred_mbf_freq =np.zeros(shape=(7,nfqs,nsta,twin))
        for ifreq in range(nfqs):
            # convert numpy array to torch tensor
            data_tt_mbf = torch.Tensor(windows_max[:,ifreq,:,:])
            # batch predict picks.
            _torch_pred_1 = pn_pnw_model(data_tt_mbf.to(device))
            _torch_pred_2 = pn_ethz_model(data_tt_mbf.to(device))
            _torch_pred_3 = pn_scedc_model(data_tt_mbf.to(device))
            _torch_pred_4 = pn_neic_model(data_tt_mbf.to(device))
            _torch_pred_5 = pn_geofon_model(data_tt_mbf.to(device))
            _torch_pred_6 = pn_stead_model(data_tt_mbf.to(device))
            _torch_pred_7 = pn_instance_model(data_tt_mbf.to(device))
            # # extract P pdf
            batch_pred_mbf_freq[0,ifreq,:, :] = _torch_pred_1[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[1,ifreq,:, :] = _torch_pred_2[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[2,ifreq,:, :] = _torch_pred_3[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[3,ifreq,:, :] = _torch_pred_4[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[4,ifreq,:, :] = _torch_pred_5[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[5,ifreq,:, :] = _torch_pred_6[1].detach().cpu().numpy()[:, :]
            batch_pred_mbf_freq[6,ifreq,:, :] = _torch_pred_7[1].detach().cpu().numpy()[:, :]


            # batch_pred_mbf=np.zeros(shape=(nsta, nfqs, twin), dtype= np.float32)
            # for ista in range(nsta):
            #     plt.plot(t-t_before,windows_std[ista,ifreq,0,:].T,linewidth=0.7)
            #     plt.plot(t-t_before,batch_pred_mbf_freq[6,:,ista, :].T*10,linewidth=0.7,color='0.8')
            #     plt.plot(t-t_before,batch_pred[6,ista, :].T*10,linewidth=0.9,color='k')
            #     plt.grid(True)
            #     # print(np.amax(batch_pred[0, :, iseg, :]))
            #     ifreq_max = np.argwhere(batch_pred_mbf_freq[0, :, ista, :] == \
            #                                 np.amax(batch_pred_mbf_freq[0, :, ista, :]))[0][0]
            #     batch_pred_mbf[0, ista, :] = batch_pred_mbf_freq[0, ifreq_max, ista, :]
            #     print(ifreq_max)
            #     plt.plot(t-t_before,batch_pred_mbf[6,ista, :].T*10,linewidth=0.9,color='r')
            
                # ifre_max = np.argmax(batch_pred[0,icha, :, iseg, :])
                # print(ifre_max)
                # batch_pred[0,icha, :, :] = batch_pred[0,icha, ifre_max, :, :]
                # plt.plot(batch_pred[0,icha, :, :])

        # batch_pred_mbf=np.zeros(shape=(nsta, nfqs, twin), dtype= np.float32)
        
        if n<=100:
            fig = plt.figure()#figsize = (11,8), dpi=200)
            fig.suptitle(str(otime)+" "+associated_volcano)
            ax = plt.subplot(1,1,1)
        for ista in range(nsta):
            if n<=100:
                ax.plot(t-t_before,data_max[ista,0,:].T,linewidth=0.7)
                ax.plot(t-t_before,batch_pred_mbf_freq[6,:,ista, :].T,linewidth=0.7,color='0.8')
                ax.plot(t-t_before,batch_pred[6,ista, :].T,linewidth=0.9,color='k')
                plt.grid(True)
            # print(np.amax(batch_pred[0, :, iseg, :]))
            ifreq_max = np.zeros(twin)
            for it in range(twin):
                # ifreq_max[it] = np.amax(batch_pred_mbf_freq[0, :, ista, it])#[0]
                batch_pred_mbf[0, ista, it] =  np.max(batch_pred_mbf_freq[0, :, ista, it])
                batch_pred_mbf[1, ista, it] =  np.max(batch_pred_mbf_freq[1, :, ista, it])
                batch_pred_mbf[2, ista, it] =  np.max(batch_pred_mbf_freq[2, :, ista, it])
                batch_pred_mbf[3, ista, it] =  np.max(batch_pred_mbf_freq[3, :, ista, it])
                batch_pred_mbf[4, ista, it] =  np.max(batch_pred_mbf_freq[4, :, ista, it])
                batch_pred_mbf[5, ista, it] =  np.max(batch_pred_mbf_freq[5, :, ista, it])
                batch_pred_mbf[6, ista, it] =  np.max(batch_pred_mbf_freq[6, :, ista, it])
            # print(ifreq_max)
            # ifreq_max = np.argwhere(batch_pred_mbf_freq[0, :, ista, :] == \
                                        # np.amax(batch_pred_mbf_freq[0, :, ista, :]))[0][0]
            # print("ifreq_max",ifreq_max)
            # batch_pred_mbf[6, ista, :] = batch_pred_mbf_freq[6, ifreq_max, ista, :]
            # print(ifreq_max)
            if n<=100:
                plt.plot(t-t_before,batch_pred_mbf[6,ista, :].T,linewidth=0.5,color='r')
        
            # ifre_max = np.argmax(batch_pred[0,icha, :, iseg, :])
            # print(ifre_max)
            # batch_pred[0,icha, :, :] = batch_pred[0,icha, ifre_max, :, :]
            # plt.plot(batch_pred[0,icha, :, :])


        if n<=100:
            pdf.savefig(fig)
        if n==100:
            pdf.close()
        # ensemble semblance
        if nsta==0:continue
        smb_pred = np.zeros([ nsta, twin], dtype = np.float32)
        smb_pred_mbf = np.zeros([ nsta, twin], dtype = np.float32)
        smb_peak = np.zeros([ nsta], dtype = np.float32)
        smb_peak_mbf = np.zeros([ nsta], dtype = np.float32)

        # Pick the phase. Assume that the PNSN is correct within 5 seconds of record.

        for ista in range(nsta): # should be 1 in this context
            
            # 0 for P-wave
            smb_pred[ ista, :] = ensemble_semblance(batch_pred[:, ista, :], paras_semblance)
            imax = np.argmax(smb_pred[ ista, t_before*40 - 5*40 : t_before*40 + 5*40 ]) # search for peak in the first 80 seconds
            if smb_pred[ ista,imax]>0:
                smb_peak[ista]=float((imax)/40)-5
            # 0 for P-wave
            smb_pred_mbf[ ista, :] = ensemble_semblance(batch_pred_mbf[:, ista, :], paras_semblance)
            imax = np.argmax(smb_pred_mbf[ ista, t_before*40 - 5*40 : t_before*40 + 5*40]) # search for peak in the first 80 seconds
            if smb_pred_mbf[ ista,imax]>0:
                smb_peak_mbf[ista]=float((imax)/40)-5



        print(smb_peak[ista],smb_peak_mbf[ista])


        # plt.show()
        del _torch_pred_1,_torch_pred_2,_torch_pred_3,_torch_pred_4,_torch_pred_5,_torch_pred_6,_torch_pred_7
        gc.collect()
        # torch.cuda.empty_cache()



        pick_error[n]=smb_peak[ista]
        pick_error_mbf[n]=smb_peak_mbf[ista]
        maxsnr[n] = max(SNR)
        whatvolcano[n]=associated_volcano
    except:
        pass
f2["pick_error"]=pick_error
f2["pick_error_mbf"]=pick_error_mbf
f2["snr"]=maxsnr
f2["associated_volcano"]=whatvolcano
f2.to_csv("../data/mbf_elep_picks_su.csv")

807903 2001-01-18T21:25:51.000000Z
Mt_St_Helens
0.325 0.25
807903 2001-01-18T21:25:52.000000Z
Mt_St_Helens
-0.35 -0.35
807903 2001-01-18T21:25:52.000000Z
Mt_St_Helens
-0.125 -0.125
807903 2001-01-18T21:25:53.000000Z
Mt_St_Helens
-0.8 -0.75
811133 2001-01-28T18:01:55.000000Z
Mt_Rainier
0.425 0.375
811133 2001-01-28T18:01:56.000000Z
Mt_Rainier
0.45 0.325
811133 2001-01-28T18:01:58.000000Z
Mt_Rainier
-0.75 -0.7
811133 2001-01-28T18:01:59.000000Z
Mt_Rainier
-0.7 -0.625
811383 2001-01-30T17:09:10.000000Z
Mt_St_Helens
-1.4 -1.5
801678 2001-02-02T14:12:50.000000Z
Mt_St_Helens
-5.0 0.425
802498 2001-02-05T09:58:32.000000Z
Mt_St_Helens
-0.425 -0.35
802498 2001-02-05T09:58:32.000000Z
Mt_St_Helens
0.425 -0.525
802498 2001-02-05T09:58:32.000000Z
Mt_St_Helens
-0.35 -0.525
804823 2001-02-12T08:29:25.000000Z
Mt_Rainier
0.95 0.8
808223 2001-02-19T23:25:06.000000Z
Mt_Rainier
-0.125 -0.275
808333 2001-02-20T02:23:26.000000Z
Mt_Rainier
0.5 0.4
809358 2001-02-21T22:39:14.000000Z
Mt_Rainier
-0.625 -0.7
809

/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55487/377156726.py:124: RuntimeWarning: invalid value encountered in divide
  data_std = crap2 / np.std(crap2) + 1e-10
/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55487/377156726.py:167: RuntimeWarning: invalid value encountered in divide
  windows_std[ista, ifreq,:, :] = _windows_mb[ista, ifreq, :]  \


803548 2001-03-07T00:53:14.000000Z
Mt_Rainier


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55487/377156726.py:222: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()#figsize = (11,8), dpi=200)


-0.1 -0.05
1465628 2001-03-20T05:17:16.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:16.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:17.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:17.000000Z
Mt_St_Helens
808348 2001-03-22T19:10:51.000000Z
Mt_Rainier
-0.375 -0.2
812068 2001-03-31T17:02:57.000000Z
Mt_St_Helens
803988 2001-04-04T14:32:12.000000Z
Mt_Rainier
807473 2001-04-11T02:32:07.000000Z
Mt_Rainier
807778 2001-04-12T11:43:14.000000Z
Mt_Rainier
-0.425 -0.275
807778 2001-04-12T11:43:16.000000Z
Mt_Rainier
-1.075 -0.65
810463 2001-04-18T00:40:42.000000Z
Mt_Rainier
-0.35 -0.175
814023 2001-04-24T12:04:05.000000Z
Mt_Rainier
-0.45 0.025
814023 2001-04-24T12:04:06.000000Z
Mt_Rainier
-0.35 -0.5
814738 2001-04-25T20:45:10.000000Z
Mt_St_Helens
-0.075 0.05
814738 2001-04-25T20:45:10.000000Z
Mt_St_Helens
0.075 0.075
814738 2001-04-25T20:45:11.000000Z
Mt_St_Helens
814738 2001-04-25T20:45:11.000000Z
Mt_St_Helens
-0.25 -0.125
814843 2001-04-25T23:13:36.000000Z
Mt_St_Helens
-0.8 -0.725
815013 2001-0

/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55487/377156726.py:124: RuntimeWarning: invalid value encountered in divide
  data_std = crap2 / np.std(crap2) + 1e-10
/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55487/377156726.py:167: RuntimeWarning: invalid value encountered in divide
  windows_std[ista, ifreq,:, :] = _windows_mb[ista, ifreq, :]  \


1.8 1.8
806018 2001-05-10T00:57:59.000000Z
Mt_St_Helens
0.425 0.425
806018 2001-05-10T00:58:00.000000Z
Mt_St_Helens
1.8 1.8
806018 2001-05-10T00:58:00.000000Z
Mt_St_Helens
-0.325 -0.4
806018 2001-05-10T00:58:00.000000Z
Mt_St_Helens
-0.025 0.15
806633 2001-05-11T19:00:09.000000Z
Mt_St_Helens
-0.175 -0.3
806633 2001-05-11T19:00:09.000000Z
Mt_St_Helens
0.05 0.125
806633 2001-05-11T19:00:10.000000Z
Mt_St_Helens
1.8 1.8
806698 2001-05-12T01:18:10.000000Z
Mt_St_Helens
1.425 1.425
806733 2001-05-12T06:07:12.000000Z
Mt_St_Helens
1.8 1.8
807373 2001-05-14T00:40:07.000000Z
Mt_St_Helens
0.65 0.425
807373 2001-05-14T00:40:08.000000Z
Mt_St_Helens
-0.65 -0.5
808058 2001-05-15T20:04:40.000000Z
Mt_St_Helens
-0.7 -1.05
808193 2001-05-16T01:49:38.000000Z
Mt_St_Helens
-5.0 -5.0
808383 2001-05-17T03:57:06.000000Z
Mt_St_Helens
0.1 0.175
808383 2001-05-17T03:57:06.000000Z
Mt_St_Helens
0.25 0.275
808633 2001-05-18T00:05:39.000000Z
Mt_St_Helens
0.125 0.125
808633 2001-05-18T00:05:40.000000Z
Mt_St_Helens
-0.45

1.8 1.8


In [ ]:
# df=pd.read_csv("../data/mbf_elep_picks_su.csv")
# df.head()
# ik=df.index[ ( df["pick_error"]!=0 ) & ( np.abs(df["pick_error"])<10 )].tolist()

# print(len(ik))
# plt.figure(figsize = (10, 5), dpi = 300)
# plt.hist(df["pick_error"][ik], 100, edgecolor = 'k', zorder = 10)
# plt.grid(True)
# std_text="STD = %2.2f"%(np.std(df["pick_error"][ik]))
# mae_text="MAE = %2.2f"%(np.mean(np.abs(df["pick_error"][ik])))
# plt.text(-6,120,std_text,fontsize=16)
# plt.text(-6,100,mae_text,fontsize=16)
# plt.xlabel("Picking errors (s)", fontsize = 20)
# plt.ylabel("Number of events", fontsize = 20)
# plt.savefig("../plots/BB_ELEP_picking_errors_highSNR.png", bbox_inches = 'tight')

In [ ]:
f2

,date,net,sta,evt_id
0,2001-01-05 13:35:05,UW,YEL,801538
1,2001-01-05 14:37:57,UW,YEL,801553
2,2001-01-18 21:25:51,UW,SEP,807903
3,2001-01-18 21:25:52,UW,YEL,807903
4,2001-01-18 21:25:52,UW,SHW,807903
...,...,...,...,...
14061,2023-07-31 14:52:37,CC,REM,3233978
14062,2023-07-31 18:10:51,CC,REM,3233998
14063,2023-07-31 18:13:22,CC,REM,3233993
14064,2023-08-01 10:56:58,CC,REM,3234208


In [ ]:
# df=pd.read_csv("../data/mbf_elep_picks_su.csv")
# df.head()
# ik=df.index[ ( df["pick_error_mbf"]!=0 ) & ( np.abs(df["pick_error_mbf"])<10 )].tolist()

# print(len(ik))
# plt.figure(figsize = (10, 5), dpi = 300)
# plt.hist(df["pick_error_mbf"][ik], 100, edgecolor = 'k', zorder = 10)
# plt.grid(True)
# std_text="STD = %2.2f"%(np.std(df["pick_error_mbf"][ik]))
# mae_text="MAE = %2.2f"%(np.mean(np.abs(df["pick_error_mbf"][ik])))
# plt.text(-4,500,std_text,fontsize=16)
# plt.text(-4,600,mae_text,fontsize=16)
# plt.xlabel("MBF Picking errors (s)", fontsize = 20)
# plt.ylabel("Number of events", fontsize = 20)
# plt.savefig("../plots/MBF_ELEP_picking_errors_allSNR.png", bbox_inches = 'tight')



# plt.figure(figsize = (10, 5), dpi = 300)
# plt.hist(df["pick_error"][ik], 100, edgecolor = 'k', zorder = 10)
# plt.grid(True)
# std_text="STD = %2.2f"%(np.std(df["pick_error"][ik]))
# mae_text="MAE = %2.2f"%(np.mean(np.abs(df["pick_error"][ik])))
# plt.text(-4,500,std_text,fontsize=16)
# plt.text(-4,600,mae_text,fontsize=16)
# plt.xlabel("BB Picking errors (s)", fontsize = 20)
# plt.ylabel("Number of events", fontsize = 20)
# plt.savefig("../plots/BB_ELEP_picking_errors_allSNR.png", bbox_inches = 'tight')


## Test to put the phase picking into a function

In [ ]:
# from mbf_elep_func import apply_mbf

# pick_error = np.zeros(len(evt_id))
# pick_error_mbf = np.zeros(len(evt_id))
# maxsnr = np.zeros(len(evt_id))
# whatvolcano = np.array(len(evt_id))

# for n in range(len(evt_id)): 
#     event_ID = str(evt_id[n])
#     otime = UTCDateTime(start_time[n])
#     networks=net[n]
#     stations=sta[n]
#     if sta[n]=="LON" or sta[n]=="LO2":continue
#     # print(net[n],sta[n])
#     try:
#         associated_volcano = df[df['Station']== sta[n]]['Volcano_Name'].values[0]
#     except: 
#         pass
#     # if associated_volcano!="Mt_Rainier":continue
            
# #################### WAVEFORM DOWNLOAD #######################
#     #Download all waveforms for that event based on stations and times
#     bulk=[[net[n], sta[n], '*', '*H*', otime-t_before_raw, otime+t_before_raw]]
#     try:
#         st = client2.get_waveforms_bulk(bulk)
#         st = resample(st,fs)  #resampling the data to 40Hz for each trace
        
#         evt_data = obspy.Stream()

#         # #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-12Hz
#         SNR,SNR_weight, no_weight,stas,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
#         for i,ii in enumerate(st):
#             network = ii.stats.network
#             station = ii.stats.station
#             ii.detrend(type = 'demean')
#             ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
#             cha = ii.stats.channel
#             starttime = ii.stats.starttime
#             signal_window = ii.copy()
#             noise_window = ii.copy()
#             # trim the data and noise window to exactly 6000 points
#             signal_window.trim(otime - t_before, otime - t_before + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
#             noise_window.trim(otime - window -t_before, otime - t_before) # noise window of the same length
#             if not len(signal_window.data) or not len(signal_window.data): continue
#             # print(len(signal_window),len(noise_window.data))
#             if not np.max(np.percentile(np.abs(signal_window.data),pr)):continue
#             if len(noise_window.data)==0:continue
#             snr = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
#                             / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
#             max_amp_time = np.argmax(noise_window.data)/fs
#             # print(snr,max_amp_time)
#             # signal_window.plot()x
#             # if snr<thr: 
#             #     st.remove(ii)
#             #     continue
#             # start the time axis 15 seconds before the pick time of the first reference station
#             t = signal_window.times()[:-1] #
#             t_diff[network+'.'+station] = starttime-otime 
#             stas.append(ii.stats.station)
#             nets.append(ii.stats.network)
#             SNR.append(snr)
#             SNR_weight.append(int(snr))
#             evt_data.append(signal_window)
            
#         sta_available,ind = np.unique(np.array(stas),return_index=True)
#         sta_available=sta_available[np.argsort(ind)]
#         if len(sta_available)==0:continue

#         print(event_ID,otime)
#         print(associated_volcano)
#         bigS = np.zeros(shape=(len(sta_available),3,6000))
#         stas=[]
#         for i in range(len(sta_available)):
#             stream = evt_data.select(station=sta_available[i])
#             # print(i,stream)
#             if len(stream[0].data)<6000:continue
#             if len(stream)<3:
#                 # copy stream to 2 components, zero the missing data.
#                 tr3 = stream[0].copy() # assumed to be the vertical
#                 tr2 = stream[0].copy();tr2.stats.channel=stream[0].stats.channel[0:2]+"N"
#                 tr1 = stream[0].copy();tr1.stats.channel=stream[0].stats.channel[0:2]+"E"
#                 tr1.data=np.zeros(len(stream[0].data))
#                 tr2.data=np.zeros(len(stream[0].data))
#                 stream=obspy.Stream(traces=[tr1,tr2,tr3])
#             # convert Stream into seisbench-friendly array    
#             # fill in big array and order data ZNE
#             bigS[i,0,:]=stream[2].data[:-1]
#             bigS[i,1,:]=stream[1].data[:-1]
#             bigS[i,2,:]=stream[0].data[:-1]
#             stas.append(sta_available[i])


#         ########### PHASE PICKING ###############
        
#         # print("put data in numpy array")

#         # allocating memory for the ensemble predictions
#         nwin,twin,nsta=bigS.shape[1],bigS.shape[-1],len(sta_available)
#         batch_pred =np.zeros(shape=(len(list_models),nsta,twin)) 
#         batch_pred_mbf =np.zeros(shape=(len(list_models),nsta,twin))


#         # evaluate
#         for imodel in list_models:
#             imodel.eval()

#         ######### Broadband workflow ################
#         crap2  = bigS.copy()
#         crap2 -= np.mean(crap2,axis=-1,keepdims=True) # demean data
#         # original use std norm
#         data_std = crap2 / np.std(crap2) + 1e-10
#         # could use max data
#         mmax= np.max(np.abs(crap2), axis=-1, keepdims=True)
#         data_max = np.divide(crap2 ,mmax,out=np.zeros_like(crap2),where=mmax!=0)
#         data_tt = torch.Tensor(data_max)
#         # batch predict picks.

#         # print("now predicting on BB")
#         for ii,imodel in enumerate(list_models):
#             batch_pred[ii,:, :]  = imodel(data_tt.to(device))[1].detach().cpu().numpy()[:, :]
                

#         ############# Multi-band Workflow ########
#         windows_std = np.zeros(shape=(nsta, nfqs, 3, twin), dtype= np.float32)
#         windows_max = np.zeros(shape=( nsta, nfqs, 3, twin), dtype= np.float32)
#         _windows = bigS.copy();#np.zeros(shape=(nsta, 3, twin), dtype= np.float32)
#         _windows_mb = np.zeros(shape=(nsta, 3, nfqs, twin), dtype= np.float32)

#         # MB filter
#         for ista in range(nsta): # loop over stations, it should be one in this benchmark test
#             for icha in range(3): # loop over channel, there should be 3 channels total
#                 _windows_mb[ista, icha, :, :] = MB_filter(_windows[ista, icha], MBF_paras)
#         _windows_mb = _windows_mb.swapaxes(1, 2)
#         # print("Shape of the multi-band data array: ",_windows_mb.shape)
#         # normalize the multi band data
#         for ista in range(nsta):
#             for ifreq in range(nfqs):
#                 # original use std norm
#                 windows_std[ista, ifreq,:, :] = _windows_mb[ista, ifreq, :]  \
#                     / np.std(_windows_mb[ista, ifreq, :]) + 1e-10
#                 # others use max norm
#                 mmax=np.max(np.abs(_windows_mb[ista, ifreq,:, :]), axis=-1, keepdims=True)
#                 windows_max[ista, ifreq,:, :] = np.divide(_windows_mb[ista, ifreq,:, :] \
#                     ,mmax,out=np.zeros_like(_windows_mb[ista, ifreq, :]),where=mmax!=0)



#         # print("now predicting on MBF")
#         batch_pred_mbf_freq =np.zeros(shape=(len(list_models),MBF_paras["nfqs"],nsta,twin))
#         for ifreq in range(MBF_paras["nfqs"]):
#             # convert numpy array to torch tensor
#             data_tt_mbf = torch.Tensor(windows_max[:,ifreq,:,:])
#             # batch predict picks.
#             for ii,imodel in enumerate(list_models):
#                 batch_pred_mbf_freq[ii,ifreq,:, :] = imodel(data_tt_mbf.to(device))[1].detach().cpu().numpy()[:, :]


#         # # batch_pred_mbf=np.zeros(shape=(nsta, nfqs, twin), dtype= np.float32)
        
#         if n<=100:
#             fig = plt.figure()#figsize = (11,8), dpi=200)
#             fig.suptitle(str(otime)+" "+associated_volcano)
#         for ista in range(nsta):
#             ax = plt.subplot(1,1,1)
#             if n<=100:
#                 ax.plot(t-t_before,data_max[ista,0,:].T,linewidth=0.7)
#                 ax.plot(t-t_before,batch_pred_mbf_freq[6,:,ista, :].T,linewidth=0.7,color='0.8')
#                 ax.plot(t-t_before,batch_pred[6,ista, :].T,linewidth=0.9,color='k')
#                 plt.grid(True)
#             # print(np.amax(batch_pred[0, :, iseg, :]))
#             for it in range(twin):
#                 for ii,imodel in enumerate(list_models):
#                     batch_pred_mbf[ii, ista, it] =  np.max(batch_pred_mbf_freq[ii, :, ista, it])
#             if n<=100:
#                 plt.plot(t-t_before,batch_pred_mbf[6,ista, :].T,linewidth=0.5,color='r')
        

#         # if n<=100:
#         #     pdf.savefig(fig)
#         # if n==100:
#         #     pdf.close()
#         # ensemble semblance
#         if nsta==0:continue
#         # smb_pred = np.zeros([ nsta, twin], dtype = np.float32)
#         # smb_pred_mbf = np.zeros([ nsta, twin], dtype = np.float32)
#         # smb_peak = np.zeros([ nsta], dtype = np.float32)
#         # smb_peak_mbf = np.zeros([ nsta], dtype = np.float32)
#         # smb_peak2 = np.zeros([ nsta], dtype = np.float32)
#         # smb_peak_mbf2 = np.zeros([ nsta], dtype = np.float32)


#         # # Pick the phase. Assume that the PNSN is correct within 5 seconds of record.
#         # sfs = MBF_paras["fs"]
#         # for ista in range(nsta): # should be 1 in this context
#         #     # 0 for P-wave
#         #     smb_pred[ ista, :] = ensemble_semblance(batch_pred[:, ista, :], paras_semblance)
#         #     imax = np.argmax(smb_pred[ ista, t_before*sfs - 5*sfs : t_before*sfs + 5*sfs ]) # search for peak in the first 80 seconds
#         #     if smb_pred[ ista,imax]>0:
#         #         smb_peak[ista]=float((imax)/sfs)-5
#         #     # 0 for P-wave
#         #     smb_pred_mbf[ ista, :] = ensemble_semblance(batch_pred_mbf[:, ista, :], paras_semblance)
#         #     imax = np.argmax(smb_pred_mbf[ ista, t_before*sfs - 5*sfs : t_before*sfs + 5*sfs]) # search for peak in the first 80 seconds
#         #     if smb_pred_mbf[ ista,imax]>0:
#                 # smb_peak_mbf[ista]=float((imax)/sfs)-5


#         # test the new function
#         smb_peak[ista],smb_peak_mbf[ista] = apply_mbf(evt_data, sta_available, \
#                 list_models, MBF_paras, paras_semblance, t_before,t_around)

#         print(smb_peak[ista],smb_peak_mbf[ista])


#         gc.collect()
#         # these are relative pick
#         pick_error[n]=smb_peak[ista]
#         pick_error_mbf[n]=smb_peak_mbf[ista]
#         # maxsnr[n] = max(SNR)
#         # whatvolcano[n]=associated_volcano
#     except:
#         pass
# # f2["pick_error"]=pick_error
# # f2["pick_error_mbf"]=pick_error_mbf
# # f2["snr"]=maxsnr
# # f2["associated_volcano"]=whatvolcano
# # f2.to_csv("../data/mbf_elep_picks_su.csv")
# pdf.close()

807903 2001-01-18T21:25:51.000000Z
Mt_St_Helens
1 Trace(s) in Stream:
UW.SEP..EHZ | 2001-01-18T21:25:36.003000Z - 2001-01-18T21:28:06.003000Z | 40.0 Hz, 6001 samples


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55449/3531268871.py:213: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  smb_peak2[ista],smb_peak_mbf2[ista] = apply_mbf(evt_data, sta_available, \


0.325 0.25
0.325 0.25
807903 2001-01-18T21:25:52.000000Z
Mt_St_Helens
1 Trace(s) in Stream:
UW.YEL..EHZ | 2001-01-18T21:25:37.003000Z - 2001-01-18T21:28:07.003000Z | 40.0 Hz, 6001 samples
-0.35 -0.35
-0.35 -0.35
807903 2001-01-18T21:25:52.000000Z
Mt_St_Helens
1 Trace(s) in Stream:
UW.SHW..EHZ | 2001-01-18T21:25:37.003000Z - 2001-01-18T21:28:07.003000Z | 40.0 Hz, 6001 samples
-0.125 -0.125
-0.125 -0.125
807903 2001-01-18T21:25:53.000000Z
Mt_St_Helens
1 Trace(s) in Stream:
UW.HSR..EHZ | 2001-01-18T21:25:38.003000Z - 2001-01-18T21:28:08.003000Z | 40.0 Hz, 6001 samples
-0.8 -0.75
-0.8 -0.75
811133 2001-01-28T18:01:55.000000Z
Mt_Rainier
1 Trace(s) in Stream:
UW.RCS..EHZ | 2001-01-28T18:01:40.000700Z - 2001-01-28T18:04:10.000700Z | 40.0 Hz, 6001 samples
0.425 0.375
0.425 0.375
811133 2001-01-28T18:01:56.000000Z
Mt_Rainier
1 Trace(s) in Stream:
UW.RCM..EHZ | 2001-01-28T18:01:41.000700Z - 2001-01-28T18:04:11.000700Z | 40.0 Hz, 6001 samples
0.45 0.325
0.45 0.325
811133 2001-01-28T18:01:58.00000

/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55449/3531268871.py:116: RuntimeWarning: invalid value encountered in divide
  data_std = crap2 / np.std(crap2) + 1e-10
/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55449/3531268871.py:144: RuntimeWarning: invalid value encountered in divide
  windows_std[ista, ifreq,:, :] = _windows_mb[ista, ifreq, :]  \


803548 2001-03-07T00:53:14.000000Z
Mt_Rainier


/var/folders/j4/3rbzxjb16kv66g_p4wkzq16h0000gn/T/ipykernel_55449/3531268871.py:166: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()#figsize = (11,8), dpi=200)


1 Trace(s) in Stream:
UW.FMW..EHZ | 2001-03-07T00:52:59.004300Z - 2001-03-07T00:55:29.004300Z | 40.0 Hz, 6001 samples
-0.1 -0.05
-0.1 -0.05
1465628 2001-03-20T05:17:16.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:16.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:17.000000Z
Mt_St_Helens
1465628 2001-03-20T05:17:17.000000Z
Mt_St_Helens
808348 2001-03-22T19:10:51.000000Z
Mt_Rainier
1 Trace(s) in Stream:
UW.FMW..EHZ | 2001-03-22T19:10:36.004600Z - 2001-03-22T19:13:06.004600Z | 40.0 Hz, 6001 samples
-0.375 -0.2
-0.375 -0.2
